# Python tutorial
[Eren Zhao](https://zhaochenyang20.github.io/about), THUCST Class 06

大家好，这篇文档主要来分享下一些 Python 的进阶内容，介绍一些实用技能栈。

我们先从最原始的一个问题出发，Python 程序如何 debug。大家之前有学习过 C++ 的话，debug 基本上依靠打断点加 print 的方法完成，Python 同样如此，我们也可以基于主流 IDE 来打断点，或者输出调试。然而我个人更倾向于将 IDE(pycharm) 视为一个带有代码补全功能的文本编辑器，所有的程序运行全是通过命令行执行。~~主要是我懒得操作 GUI 来配环境。~~

我来解释下为什么我会喜欢纯粹用命令行而不是用 IDE 的 GUI。这当然是从 python 的环境需要 conda 支持出发的，而 conda 的曹组，IPython 等等完全是在命令行进行的。为什么会使用 conda，此处不再赘述，请查看讲义即可。此处展示下我电脑里的 conda 环境：

In [ ]:
!source activate
!conda info -e

# conda environments:
#
base                     /Users/zhaochen20/opt/anaconda3
analysis              *  /Users/zhaochen20/opt/anaconda3/envs/analysis
torch                    /Users/zhaochen20/opt/anaconda3/envs/torch



主要是我的电脑里 conda 环境各不相同，而每次为新的工程配置好了 conda 环境后，再配置到 IDE 里比较麻烦，所以我所有的代码都是 IDE 编辑 + shell 执行。

我这么做就会遇到一个小问题，我该怎么打断点呢，听上比较麻烦，似乎没有像 C艹 一样很方便的打断点的工具。这个时候，第一个库，IPython 可以起到很大作用。

IPthon (注意大小写)是一个基于 Python Shell 的交互式解释器，但是有比默认Shell强大得多的编辑和交互功能。同时，还是一个功能强大的 Python 库。IDE 的断点功能就是依靠 IPython 的 embed 方法实现的，我这里演示下。

embed 的功能为，当运行到这一指令时，进入 IPython 交互环境，同时监测所有的变量，将所有的修改进入内存当中，退出这一环境之后再接着运行余下的程序。

In [1]:
from IPython import embed

x = 1
embed(header = "x")

Python 3.8.12 (default, Oct 12 2021, 06:23:56) 
Type 'copyright', 'credits' or 'license' for more information
IPython 8.4.0 -- An enhanced Interactive Python. Type '?' for help.


x




这里其实就是起到了打断点的作用，基于此，我们就可以来讨论 Python 工程 debug 的问题。
最原始的办法当然是直接跑，然后全局跑，直到出 bug，然后对着行数去 debug。关键是你一次不一定能对，这样子得反复折腾很久。甚至，你可能会跑一个特别大的程序，中途不序列化储存数据，每次重跑都会跑很久，这显然很 brute 而且低效。
我自己的 debug 方法如下：

In [3]:
from IPython import embed

try:
    x += 1
except Exception as e:
    embed()

每当我对某段代码感到不确定，比如担心读取 json 的时候出现解码错误，大概用这个流程：

In [4]:
from IPython import embed

@metric
def prettify(file):
    with open(file, "r", encoding="utf-8") as f:
        try:
            meta_web = json.load(f)["html_content"]
        except Exception as e:
            print(e)
            print(file)
            embed()
            return
        soup = BeautifulSoup(meta_web, "lxml").prettify()
        with open(test_txt, "w", encoding = "utf-8") as t:
            t.write(soup)

NameError: name 'metric' is not defined

基于这部分代码，我们能够实现读取文件，倘若文件无法去按照 json 格式读取，进入 exception 逻辑，打印出问题和出现问题的文件，进入 IPython 交互界面，根据需求查看程序执行情况，选择退出或者返回。

接下来，讲述下 jupyter notebook 的使用。实际上，这个讲义就是按照 jupyter notebook 写的，之前我一直对 jupyter 不以为然，直到最近才发现他真香。
> Jupyter Notebook 以网页的形式打开格式文件，可以在网页页面中直接编写代码和运行代码，代码的运行结果也会直接在代码块下显示的程序。如在编程过程中需要编写说明文档，可在同一个页面中直接编写，便于作及时的说明和解释。
> 同时，jupyter 能够在远端服务器启动后，在本地的浏览器编写，相当于是在 浏览器里的远端 IDE。
> 他还支持导出 tex, pdf, md 等主流文件，非常方便。

这里附带上一些我用 jupyter 写出的成果：
[Distribution is all you need PDF](https://zhaochenyang20.github.io/pdf/distribution.pdf)
[Distribution is all you need IPYNB](https://zhaochenyang20.github.io/ipynb/distribution.ipynb)
[Python Tutorial PDF](https://zhaochenyang20.github.io/pdf/python_tutorial.pdf)
[Python Tutorial IPYNB](https://zhaochenyang20.github.io/ipynb/python_tutorial.ipynb)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import operator as op
from functools import reduce
def const(n, r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer / denom
def binomial(n, p):
    q = 1 - p
    y = [const(n, k) * (p ** k) * (q ** (n-k)) for k in range(n)]
    return y, np.mean(y), np.std(y)
for ls in [(0.5, 40)]:
    p, n_experiment = ls[0], ls[1]
    x = np.arange(n_experiment)
    y, u, s = binomial(n_experiment, p)
    plt.scatter(x, y, label=r'$\mu=%.2f,\ \sigma=%.2f$' % (u, s))
plt.legend()
plt.show()

# 常用基础库

## tqdm

接下来是 tqdm 和 p_tqdm，这两个库功能简单，直观易用，就是为循环加入可视化进度条，便于监视程序执行进度。

In [2]:
from tqdm import tqdm

for i in tqdm(range(10000000)):
    pass

100%|██████████| 10000000/10000000 [00:01<00:00, 6385419.04it/s]


这当我跑小型循环的时候，他当然不太需要监督进度，但是比如这次人智导要处理上千个文件，处理过程大概需要 30 分钟，故而不加入进度条，我们很难看到代码运行到了哪一步，这时候 tqdm 就会很方便。
下面是 p_tqdm，p 是 parallel 的意思。使用单个 tqdm 对于 cpu 的利用率不会很高，采用并行多线程，会显著提高效率。

In [ ]:
from p_tqdm import p_map
from IPython import embed

def add(list_item, arg):
    return list_item + arg

origin_list = range(10000)
try:
    results = p_map(add, origin_list, range(1, len(origin_list) + 1))
    embed()
except Exception as e:
    embed()

In [ ]:
from p_tqdm import p_map

def train_each_document(document, process_id):
    """
    :param document: a json file path, containing a long string like "苟利国家生死以|美国的华莱士比你们不知道高到哪里去了|没这个能力|"
    :return: the neuron itself
    """
    neuron = Neuron()
    try:
        with open(document, "r", encoding="utf-8", errors="ignore") as f:
            contents = json.loads(f.read())
    except:
        return neuron
    string_list = contents.split("|")
    for sentence in string_list:
        neuron.train_each_sentence(sentence)
    store_path = Path.cwd() / "trans_training_result"
    if not store_path.is_dir():
        os.makedirs(store_path)
    store_name = store_path / f"{process_id}.npz"
    np.savez(store_name, neuron.one, neuron.two, neuron.three)
    return neuron

def train_total_dir(self, director):
        """
        use a director of document to train the neuron
        """
        training_list = get_list(director)
        results = p_map(train_each_document, training_list, range(1, len(training_list) + 1))
        for each in results:
            self.add_neuron(each)

## typing

typing 是一个用于写出精准注释的库，正常情况下，我们的代码需要如下的注释：

In [ ]:
def train_each_document(document: str, process_id: int):
    """
    :param document: a json file path, containing a long string like "苟利国家生死以|美国的华莱士比你们不知道高到哪里去了|没这个能力|"
    :return: the neuron itself
    """

typing 库还支持更高级的注释，比如 List（注意大小写）：

In [ ]:
from typing import List
def print_names(names: List[str]) -> None:
    for student in names:
        print(student)

当然，也支持[注释自己定义的类](https://zhaochenyang20.github.io/2022/01/09/CS/others/typing/)。可能在 Django 里有些作用...

## Counter and OrderedDict

Counter 和 OrderedDict 是两个 dictionary 的子类，非常方便。
我们都知道在字典中查找不存在的键，程序会抛出 KyeError的异常，但是由于 Counter 用于统计计数，因此 Counter 不同于字典，如果在 Counter 中查找一个不存在的元素，不会产生异常，而是会返回 0，这其实很好理解，Counter 计数将不存在元素的 count 值设置为 0。

对我而言，用了 Counter 类可以大量节省这个语句：

In [ ]:
from collections import Counter
try:
   course_total_order[course] += 1
except:
   course_total_order[course] = 1

# 用 Counter 类之后，只用写
course_total_order[course] += 1

OrderedDict 顾名思义，有序字典，Python 原生字典的实现是有序的，但是这个顺序很复杂，大概率不同于 key-value pair 加入 Dict 的顺序，而 OrderedDict 可以按照加入顺序来遍历。

In [1]:
from collections import OrderedDict

dict = OrderedDict()

dict["Bob"] = 1
dict["Alice"] = 2
dict["Carl"] = 3
    
for key in dict:
    print(key, dict[key])

Bob 1
Alice 2
Carl 3


## Pathlib

接下来是 Pathlib，我之前在我的 Mac 上写工程，我的文件路径是硬编码的，导致移植到 windows 上会出很多问题，而且经常因为路径里的"/"和"\"被坑，直到 lambda 给我推荐了 Pathlib

Pathlib 和 os 的很多操作类似，但是封装的更好，比如快捷的工作路径切换，目录拼接，还有文件检测。

In [1]:
import os
from pathlib import Path

store_path = Path.cwd() / "final_training_result"
if not store_path.is_dir():
    # os.makedirs(store_path)
    pass
store_name = store_path / "refactor.npz"
print(store_name, store_path)

/Users/zhaochen20/blog_zhaochen20/source/ipynb/final_training_result/refactor.npz /Users/zhaochen20/blog_zhaochen20/source/ipynb/final_training_result


## Numpy 序列化

Numpy 的功能非常强大，是专业的数据科学库。很多人说，numpy 就是用 C++ 写的 Python 库，效率堪比 C++。实际上，具体去了解下的话，numpy 比 C++ 还底层，他实际上大量用了 Fortran 来编写底层计算，堪比汇编的效率，这些内容留待数据分析课讲解，这里先讲讲序列化与反序列化。

在大家的小学期，大多数同学选择把一个很大的 Dict/list 存为 json，然后读取 json。我的建议是，用 numpy 把 Counter 序列化为 npz 格式，然后读取 npz。json 实际上是文本文件，Linux 内核读取文本文件的速度远低于读取二进制文件，而 npz（~~还有 npy~~）实际上是二进制文件，读取和加载速度非常快，而且比文本文件内存少了很多，从 1.4G 的 json 压缩为 0.6G 的npz。

当然，npz 自然有缺点，只能用 numpy 来读取，而且人类不可理解。我 SRT 的工作，需要把爬虫爬下来的网页解析 html，然后保存。我自己肯定就存 npz 了，但是我们组里其他同学完全不会用，所以我还是存了 json 和 CSV。

In [6]:
import os
from pathlib import Path
import numpy as np

store_path = Path.cwd() / "test.npz"
dict = {"Eren zhao": 1, "跳跳鸟": 2, "鲁大师": 3, "lambda": 4, "c7w": 5}
np.savez(store_path, dict=dict)

In [8]:
!ls

Python tutorial.ipynb distribution.ipynb    test.npz


In [11]:
dictionary = np.load(Path.cwd() / "test.npz", allow_pickle=True)["dict"].item()
print(dictionary)

{'Eren zhao': 1, '跳跳鸟': 2, '鲁大师': 3, 'lambda': 4, 'c7w': 5}


当然，这背后还涉及 numpy Ndarray 的取对象和切片问题，需要另外研究。

## decorator

decorator 并非一个库，而是 python 的装饰器类的集合，与 OOP 的装饰器模式一脉相承，具体内容较为复杂，推荐大家阅读[Clean Python 这本书](https://zhaochenyang20.github.io/pdf/clean%20python.pdf)可以深入理解。
我实验室的大师兄（~~还有掌门师姐~~）他们写的 code，满篇都是这个样子：

In [ ]:
@metric
def train_total_dir(self, director):
    """
    use a director of document to train the neuron
    """
    try:
        training_list = get_list(director)
        results = p_map(train_each_document, training_list, range(1, len(training_list) + 1))
        for each in results:
            self.add_neuron(each)
        neo_one = {}
        neo_two = {}
        neo_three = {}
        print("filter 1 unit")

我一直不太理解，这个 @metric 是什么意思，后来我发现了这个是修饰器语法，和 shebang 语法很像。（shebang 待会再讲）

In [ ]:
@metric
def func():
    pass

# 这句话等价于

fuc = metric(func)

然后，metric 修饰器的定义自行查找，一般而言，修饰器不影响代码逻辑，但是可以添加统一的功能，类似于打印执行的时间和运行在哪张卡上，其实就是 oop 的封装思想。

In [ ]:
def metric(fn):
    """running time for each main function"""

    @functools.wraps(fn)
    def wrapper(*args, **kw):
        print('start executing %s' % (fn.__name__))
        start_time = time.time()
        result = fn(*args, **kw)
        end_time = time.time()
        t = 1000 * (end_time - start_time)
        print('%s executed in %s ms' % (fn.__name__, t))
        return result
    return wrapper

这是我常用的装饰器，展示程序运行的毫秒数。

## argparase
接下来是 argparase，广泛应用于形式化解析命令行参数。

In [ ]:
import argparse

def parser_data():
    parser = argparse.ArgumentParser(
        prog='Pinyin Input Method',
        description='Pinyin to Chinese.',
        allow_abbrev=True,
    )
    parser.add_argument('-i', '--input-file', dest='input_file_path', type=str, help="Input file")
    parser.add_argument('-o', '--output-file', dest='output_file_path', type=str, help="Output file")
    parser.add_argument('-c', '--coefficient', dest='coefficient', type=float, nargs=2, default=[0.4, 0.5], help="coefficient")
    input_file_path = parser.parse_args().input_file_path
    output_file_path = parser.parse_args().output_file_path
    coefficient = parser.parse_args().coefficient
    try:
        assert os.path.exists(input_file_path) == True
    except:
        print(f"You may use an existing file. But you have use an unexisting file: {input_file_path}")
        print("Thus, the progress would exit right now.")
        exit(1)
    try:
        assert len(coefficient) == 2 and coefficient[0] <= 1 and coefficient[1] <= 1
    except:
        print(f"You may input two coefficient. And theyshould be less than 1. But you have input: {coefficient}")
        print("Thus, the progress would exit right now.")
        exit(1)
    return input_file_path, output_file_path, coefficient

运行方式 'python3 pinyin.py -i  /Users/zhaochen20/THU_CST/2022_spring/人智导/作业/input-method/测试语料/input_2.txt -o ./test.txt -c 1 1' 

 可以通过 'python3 pinyin.py -h` 获取帮助。当然，还有如何输入布尔变量的问题，这个比较复杂，具体内容可以参考我 Notion 上的 [Python Tutorial 2](https://zhaochen20.notion.site/Python-Part-2-dc4ac581989c4533894bc68a83b0a8d9)

# Linux

接下来讲点 Linux，主要是方便大家在服务器上跑代码...
说起来，我之所以选择用 jupyter 跑远程代码，是应为我的 Mac 各种 IDE 连接服务器都很拉胯，我试过 Vscode，Jetbrain Gateaway...
也用过原生 ssh + nano，直到一位朋友给我推荐了 jupyter，我一开始以为 jupyter 的意义是提供了代码 + 文档的共同编写环境，实际上：

> jupyter是懒人包，方便可视化和debug，尤其是服务器上。web环境调ui比xwindow方便。你设置好以后在自己笔记本上用 ip 访问和操作，比较方便。

简直是神器！

## heredoc

稍微说一句 heredoc

In [ ]:
> cat << EOF > now
heredoc> #! usr/local/bin/python3.9
heredoc> from datetime import datetime
heredoc> print('current time is %s' % datetime.now())
heredoc> EOF
# 这里需要在命令行运行，不做展示

heredoc 就是类似上方的格式，可以实现创建新文本的功能，虽然用 vim，nano，touch 都可以。~~反正技多不压身~~

## shebang

接下来是 shebang:

> 在计算领域中，Shebang（也称为Hashbang）是一个由井号和叹号构成的字符序列 `#!`，其出现在文本文件的第一行的前两个字符。 在文件中存在 Shebang 的情况下，类 Unix 操作系统的程序加载器会分析 Shebang 后的内容，将这些内容作为解释器指令，并调用该指令，并将载有 Shebang 的文件路径作为该解释器的参数。


给个例子：

In [3]:
#! usr/local/bin/python3.9
print("hello shebang")

hello shebang


## conda

conda 是最后一个内容。~~相信大家都懂，我就不讲了~~

In [ ]:
# 一个基本的conda流程
> conda create -n env_name python=3.8
> source activate env_name
> conda install whaterever
# 如果conda装不了，那就用pip
> pip install whatever

# conda activate 失败
> conda activate SRT_crawler
CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
> source activate
> conda activate SRT_crawler

#  在某个具体的 conda 环境下，使用 pip 就是安装到这个特定的 conda 环境
# 对于单个包的安装，可能遇上安装超时的问题
> pip install python-moudle

raise ReadTimeoutError(self._pool, None, "Read timed out.")
pip._vendor.urllib3.exceptions.ReadTimeoutError: HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Read timed out.

# 解决方案
> pip --timeout=100 install python-moudle

# 如果是依靠某个具体的requirement.txt，则：
pip --default-timeout=100 install -r requirements_demo.txt

# ok，太慢，不如直接换源
> conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free/
> conda config --set show_channel_urls yes

最后讲一讲 pythonic 代码风格的问题，老实说，我不是很确定我是否 pythonic，pythonic 的精华很大部分来自各种推导，比如这一段：

In [ ]:
# 出自软工后端小作业，写的真的喵！
def messages_for_user(request):

    def gen_response(code: int, data: str):
        return JsonResponse({
            'code': code,
            'data': data
        }, status=code)

    try:
        assert request.method == "POST"
        name = request.COOKIES['user']
        user = User.objects.get(name=name)
        messages = Message.objects.filter(user=user)
        assert len(messages) != 0
        return gen_response(200, [
            {
                'title': msg.title,
                'content': msg.content,
                'timestamp': int(msg.pub_date.timestamp())
            }
            for msg in messages.order_by('-pub_date')
        ])
    except Exception as e:
        print(e)
        return gen_response(400, "查无此人")

我这里讲讲我们实验室的代码风格，自己很喜欢。
一个 Python 工程分若干个脚本文件，每个文件构造如下：

1. 引用原生库和手写库
2. 定义全局变量
3. 定义修饰器
4. 定义类和对象函数
5. 定义函数（包括 argparse）
6. 定义主函数

最后定义 pipeline，用 os.system 一个个跑，比如这亚子：

In [ ]:
import os

def pipeline():
    print("refactor start")
    os.system("python3 refactor_data.py > refactor_log.txt")
    print("train start")
    os.system("python3 train.py -s Large > training_log.txt")
    print("judge start")
    os.system("./complete.sh > complete_log.txt")

if __name__ == "__main__":
    pipeline()

这里实际上是用 os 代替了 ShellScript，实际上所有的 shell 脚本都能用 os.system 代替，而且更可读，毕竟大家更了解 Python，而非 shell 语法。虽然 shell 语法个人认为不太重要，但是 linux 指令很重要。

```
for i in 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1
do
	for j in 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1
	do
		python3 pinyin.py -i ./测试语料/input_2.txt -o ./test.txt -c $i $j
	done
done
for i in 0.9 0.99 0.999 0.9999 0.99999 1
do
        for j in 0.9 0.99 0.999 0.9999 0.99999 1
        do
                python3 pinyin.py -i ./测试语料/input_2.txt -o ./test.txt -c $i $j
        done
done
```